In [7]:
from safetensors.torch import load_file
from collections import defaultdict
import torch 
import json 
bp = "/workspace/SD-lcm-tpu/model_export/tmp/picxreal3_sd15_sd15/unet/path_to_unet_no_cfg.bmodel"
sp = "/workspace/SD-lcm-tpu/model_export/safetensors/sd/picxreal.safetensors"
def reorder_shape_compute(shape1, dtype_type = 2):
    oc, ic, kh, kw = shape1
    IC_PARALLEL = 64 // dtype_type
    kernel_hw = kh * kw
    new_ic = (ic+IC_PARALLEL-1) // IC_PARALLEL
    new_hw = kernel_hw * IC_PARALLEL
    if new_ic * new_hw > 65535 and len(shape) == 4:
        shape = [1, oc, new_ic, new_hw]
    else:
        shape = [1, oc, 1, new_ic * new_hw]
    return shape
    
def convert_shape(cur_shape, dtype_type=2):
    if len(cur_shape) == 4:
        cur_shape = reorder_shape_compute(cur_shape, dtype_type)
    cur_shape.sort()
    res = [i for i in cur_shape if i != 1]
    return tuple(res)
data = load_file(sp)
names = [i for i in data.keys() if "model.diffusion_model" in i] 
data_list = [
t = {k:[convert_shape(list(data[k].shape)), data[k].mean().item(), data[k].min().item(), data[k].max().item()] for k in names}
names

['model.diffusion_model.input_blocks.0.0.bias',
 'model.diffusion_model.input_blocks.0.0.weight',
 'model.diffusion_model.input_blocks.1.0.emb_layers.1.bias',
 'model.diffusion_model.input_blocks.1.0.emb_layers.1.weight',
 'model.diffusion_model.input_blocks.1.0.in_layers.0.bias',
 'model.diffusion_model.input_blocks.1.0.in_layers.0.weight',
 'model.diffusion_model.input_blocks.1.0.in_layers.2.bias',
 'model.diffusion_model.input_blocks.1.0.in_layers.2.weight',
 'model.diffusion_model.input_blocks.1.0.out_layers.0.bias',
 'model.diffusion_model.input_blocks.1.0.out_layers.0.weight',
 'model.diffusion_model.input_blocks.1.0.out_layers.3.bias',
 'model.diffusion_model.input_blocks.1.0.out_layers.3.weight',
 'model.diffusion_model.input_blocks.1.1.norm.bias',
 'model.diffusion_model.input_blocks.1.1.norm.weight',
 'model.diffusion_model.input_blocks.1.1.proj_in.bias',
 'model.diffusion_model.input_blocks.1.1.proj_in.weight',
 'model.diffusion_model.input_blocks.1.1.proj_out.bias',
 'model

In [8]:
from newtool import *
from untool import *
model = EngineOV(bp, pre_malloc=False)

In [9]:

def make_torch2c(tensor):
    ptr = tensor.data_ptr()
    return ctypes.c_void_p(ptr)

def read_tensor_to_torch_by_mem(address, shape, dtype=torch.bfloat16, from_json=False):
    address -= 4294967296 
    address += 0 if not from_json else bm.bmodel_coeff_start
    data = torch.ones(shape, dtype=dtype)
    size = data.numel() * data.element_size()
    read_binary_by_address( bm.model_ctx, address, size, make_torch2c(data))
    return data 
    
class ModelBinary:
    
    def __init__(self, model_coeff_info, stage_idx=0):
        self.model_coeff_info   = model_coeff_info
        self.model_ctx          = model_coeff_info['model_ctx']
        self.bmodel_coeff_start = None
        self.all_coeff          = None
        self.coeff_size         = None
        self.read_bmodel_with_mem(stage_idx)
        
    def read_bmodel_with_mem(self, stage_idx=0):
        self.all_coeff          = get_all_coeff(self.model_ctx, stage_idx)
        self.bmodel_coeff_start = self.model_coeff_info['stage_info'][stage_idx]['coeff']['start']
        self.coeff_size         = self.model_coeff_info['stage_info'][stage_idx]['coeff']['size']
    
    def read_tensor_by_address(self, address, shape, dtype=torch.float16, from_json=True):
        if from_json:
            address = address - MLIR_START + self.bmodel_coeff_start
        data = torch.ones(shape, dtype=dtype)
        size = data.numel() * data.element_size()
        read_binary_by_address( self.model_ctx, address, size, make_torch2c(data))
        return data 
    
    def read_tensor_by_address_with_mem(self, address, shape, dtype=torch.float16, from_json=False):
        if from_json:
            address = address - MLIR_START + self.bmodel_coeff_start
        else:
            address = address - MLIR_START
        data = torch.ones(shape, dtype=dtype)
        size = data.numel() * data.element_size()
        read_binary_by_address_with_mem( self.all_coeff, address, size, make_torch2c(data))
        return data
    
    def write_tensor_by_address_with_mem(self, address, size, data):
        write_binary_by_address_with_mem(self.all_coeff, address, size, data)

bm = ModelBinary(model.model_info)

In [10]:

jp = "/workspace/SD-lcm-tpu/model_export/tmp/picxreal3_sd15_sd15/unet/path_to_unet_no_cfg.bmodel.json"
jd = json.load(open(jp,"r"))
start = 4294967296
end = 4294967296 + bm.coeff_size
dtype_map = {"f16":torch.float16, "bf16":torch.bfloat16, "f32":torch.float32}

def parse_shape_dtype(x):
    ts = x[1:-1].split("x")
    dtype = dtype_map[ts[-1]]
    shape = [int(i) for i in ts[:-1] if "1" != i]
    shape.sort()
    return tuple(shape), dtype

res = []
nres = []
have = set()
for i in jd:
    for op in i['operands']:
        if "address" in op and op['address'] >= start and op['address'] < end and op['address'] not in have:
            have.add(op['address'])
            shape, dtype = parse_shape_dtype(op['memory_type'])
            res.append([op['address'],shape, dtype,op['name']])
res.sort(key = lambda x:x[0])

```
 'model.diffusion_model.time_embed.0.bias',
 'model.diffusion_model.time_embed.0.weight',
 'model.diffusion_model.time_embed.2.bias',
 'model.diffusion_model.time_embed.2.weight'
```

In [21]:
get_tensor = lambda x: bm.read_tensor_by_address(x[0], x[1], x[2], True)
def get_info(x):
    t = get_tensor(x)
    return t.mean().item(), t.min().item(), t.max().item()

In [17]:
source_shape_info = defaultdict(list)
for k,v in t.items():
    source_shape_info[v[0]].append([v[1:], k])

[[4294967296, (160,), torch.bfloat16, 'emb0.1748_folder_bf16'],
 [4294971392, (320, 640), torch.bfloat16, 'input0.1input0.1_filter_bf16_1_0'],
 [4295380992,
  (320, 640),
  torch.bfloat16,
  'input0.1input0.1_filter_bf16_1_640'],
 [4295790592, (320, 1280), torch.bfloat16, 'input0.1input0.1_filter_bf16'],
 [4296609792, (640,), torch.bfloat16, 'input0.1707_bf16_0_0'],
 [4296613888, (640,), torch.bfloat16, 'input0.1707_bf16_0_640'],
 [4296617984, (1280,), torch.bfloat16, 'input0.1707_bf16'],
 [4296622080, (640, 1280), torch.bfloat16, 'input2.1input2.1_filter_bf16_1_0'],
 [4298260480,
  (640, 640),
  torch.bfloat16,
  'input2.1input2.1_filter_bf16_1_640_0_0'],
 [4299079680,
  (640, 640),
  torch.bfloat16,
  'input2.1input2.1_filter_bf16_1_640_0_640'],
 [4299898880, (1280, 1280), torch.bfloat16, 'input2.1input2.1_filter_bf16'],
 [4303175680, (640,), torch.bfloat16, 'input2.1705_bf16_0_0'],
 [4303179776, (640,), torch.bfloat16, 'input2.1705_bf16_0_640_0'],
 [4303183872, (1280,), torch.bfloat

In [29]:
bmodel_shape_info = defaultdict(list)
bmodel_map = {}
for item in res:
    temp_id = get_info(item)
    bmodel_shape_info[item[1]].append([*temp_id, item[-1]])
    bmodel_map[item[-1]] = [item[1],*temp_id]

In [150]:
match_mem = defaultdict(dict)
def is_match_item(source_name, bmodel_name, tolerance=0.01):
    if bmodel_name in match_mem[source_name]:
        return match_mem[source_name][bmodel_name]
    source = t[source_name]
    bmodel = bmodel_map[bmodel_name]
    # reorder in bmodel whether to match mean value 
    for i in range(3):
        if i == 0 and "reorder" in bmodel_name:
            continue
        if abs(bmodel[i+1] - source[i+1]) > tolerance:
            match_mem[source_name][bmodel_name]=False
            return False
    match_mem[source_name][bmodel_name] = True
    return True
match_table = []
vis_mem = set()
not_match = []
def match(source_shape_info, bmodel_shape_info):
    for sname in tqdm(t.keys()):
        flag=  False
        for bname in bmodel_map.keys():
            if bname in vis_mem: continue
            if is_match_item(sname, bname):
                vis_mem.add(bname)
                match_table.append([sname, bname])
                flag = True
                break
        if not flag:
            not_match.append(sname)
match(source_shape_info, bmodel_shape_info)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 686/686 [00:00<00:00, 3737.62it/s]


In [151]:
len(match_table)

656

In [152]:
len(not_match)

30

In [153]:
not_match

['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.ff.net.0.proj.bias',
 'model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn2.to_k.weight',
 'model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn1.to_q.weight',
 'model.diffusion_model.input_blocks.5.1.transformer_blocks.0.attn2.to_out.0.weight',
 'model.diffusion_model.input_blocks.5.1.transformer_blocks.0.ff.net.0.proj.bias',
 'model.diffusion_model.input_blocks.7.1.transformer_blocks.0.ff.net.0.proj.bias',
 'model.diffusion_model.input_blocks.7.1.transformer_blocks.0.ff.net.0.proj.weight',
 'model.diffusion_model.input_blocks.8.1.transformer_blocks.0.ff.net.0.proj.bias',
 'model.diffusion_model.middle_block.1.transformer_blocks.0.ff.net.0.proj.bias',
 'model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn2.to_out.0.weight',
 'model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn2.to_v.weight',
 'model.diffusion_model.output_blocks.10.1.transformer_blocks.0.ff.net.0.proj.bi

In [82]:
import re
def handle_head_bname(s):
    return re.sub(r"head_\d","",s)

In [83]:
head_list = []
head_map  = defaultdict(list)
for k in bmodel_map:
    if "head" in k:
        head_list.append(k)
        head_map[handle_head_bname(k)].append(k)

In [168]:
sorted(head_list)

['input10.1_head_0_bias_reorder',
 'input10.1_head_0_filter_reorder',
 'input10.1_head_0input10.1_filter_weight_head_0_bf16',
 'input10.1_head_1_filter_reorder',
 'input10.1_head_1input10.1_filter_weight_head_1_bf16',
 'input10.1_head_2_filter_reorder',
 'input10.1_head_2input10.1_filter_weight_head_2_bf16',
 'input10.1_head_3_filter_reorder',
 'input10.1_head_3input10.1_filter_weight_head_3_bf16',
 'input10.1_head_4_filter_reorder',
 'input10.1_head_4input10.1_filter_weight_head_4_bf16',
 'input10.1_head_5_filter_reorder',
 'input10.1_head_5input10.1_filter_weight_head_5_bf16',
 'input10.1_head_6_filter_reorder',
 'input10.1_head_6input10.1_filter_weight_head_6_bf16',
 'input10.1_head_7_filter_reorder',
 'input10.1_head_7input10.1_filter_weight_head_7_bf16',
 'input14.1_head_0_bias_reorder',
 'input14.1_head_0_filter_reorder',
 'input14.1_head_0input14.1_filter_weight_head_0_bf16',
 'input14.1_head_1_filter_reorder',
 'input14.1_head_1input14.1_filter_weight_head_1_bf16',
 'input14.1_

In [169]:
len(head_map)

60

In [155]:
def prod(x):
    res = 1
    for i in x: res*=i
    return res
def handle_head_info(i,v):
    res = [[],0,0,0,i]
    shapes = [bmodel_map[k][0] for k in v]
    res[0] = [shapes]
    res[1] = 0
    res[2] = min(bmodel_map[k][2] for k in v)
    res[3] = max(bmodel_map[k][3] for k in v)
    return res
head_map_fix = {
    i:handle_head_info(i,v) for i,v in head_map.items()}

In [170]:
bmodel_map['input237.1_head_0_bias_reorder']

[(640,), -0.000370025634765625, -0.94921875, 0.73046875]

In [174]:
head_match_mem = set()
head_match_table = []
head_not_match = []
for not_match_k in tqdm(t.keys()):
    flag = False
    for k in head_map_fix.keys():
        if k in head_match_mem:
            continue
        if abs(t[not_match_k][2] - head_map_fix[k][2]) > 0.01:
            continue
        if abs(t[not_match_k][3] - head_map_fix[k][3]) > 0.01:
            continue
        head_match_table.append([not_match_k, k])
        head_match_mem.add(k)
        flag = True
        break
    if not flag:
        head_not_match.append(not_match_k)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 686/686 [00:00<00:00, 31563.45it/s]


In [175]:
len(head_match_table)

60

In [165]:
sorted(head_not_match)

['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.ff.net.0.proj.bias',
 'model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn1.to_q.weight',
 'model.diffusion_model.input_blocks.5.1.transformer_blocks.0.ff.net.0.proj.bias',
 'model.diffusion_model.input_blocks.7.1.transformer_blocks.0.ff.net.0.proj.bias',
 'model.diffusion_model.input_blocks.7.1.transformer_blocks.0.ff.net.0.proj.weight',
 'model.diffusion_model.input_blocks.8.1.transformer_blocks.0.ff.net.0.proj.bias',
 'model.diffusion_model.middle_block.1.transformer_blocks.0.ff.net.0.proj.bias',
 'model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn2.to_v.weight',
 'model.diffusion_model.output_blocks.10.1.transformer_blocks.0.ff.net.0.proj.bias',
 'model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn1.to_q.weight',
 'model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn2.to_q.weight',
 'model.diffusion_model.output_blocks.4.1.transformer_blocks.0.ff.net.0.proj.bias',

In [163]:
len(head_map_fix)

60

In [ ]:
def recover_split_from_attention(bmodel_map):
    if "head" in 
    pass

In [ ]:
bmodel_map[

In [86]:
bmodel_map["input10.1_head_2input10.1_filter_weight_head_2_bf16"]

[(40, 320), -0.000164031982421875, -0.1826171875, 0.26171875]

In [84]:
head_map

defaultdict(list,
            {'input10.1_input10.1_filter_weight__bf16': ['input10.1_head_0input10.1_filter_weight_head_0_bf16',
              'input10.1_head_1input10.1_filter_weight_head_1_bf16',
              'input10.1_head_2input10.1_filter_weight_head_2_bf16',
              'input10.1_head_3input10.1_filter_weight_head_3_bf16',
              'input10.1_head_4input10.1_filter_weight_head_4_bf16',
              'input10.1_head_5input10.1_filter_weight_head_5_bf16',
              'input10.1_head_6input10.1_filter_weight_head_6_bf16',
              'input10.1_head_7input10.1_filter_weight_head_7_bf16'],
             'input10.1__filter_reorder': ['input10.1_head_0_filter_reorder',
              'input10.1_head_1_filter_reorder',
              'input10.1_head_2_filter_reorder',
              'input10.1_head_3_filter_reorder',
              'input10.1_head_4_filter_reorder',
              'input10.1_head_5_filter_reorder',
              'input10.1_head_6_filter_reorder',
              

In [48]:
not_match

['model.diffusion_model.output_blocks.11.0.emb_layers.1.weight',
 'model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn1.to_k.weight',
 'model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn1.to_out.0.weight',
 'model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn1.to_q.weight',
 'model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn1.to_v.weight',
 'model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn2.to_out.0.weight',
 'model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn2.to_q.weight',
 'model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn1.to_k.weight',
 'model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn1.to_out.0.weight',
 'model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn1.to_q.weight',
 'model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn1.to_v.weight',
 'model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn2.to_out.0.weight',
 'model.diffusion

权重这里分为了  
1. reordered
2. folder
    - emb0.1748_folder_bf16 
4. split 

4294971392,4295380992   model.diffusion_model.time_embed.0.weight 

In [22]:
res[25:35]

[[4306067456, [320, 2880], torch.bfloat16, 'hidden_states2.1_filter_reorderd'],
 [4307910656, [320], torch.float32, '689'],
 [4307914752, [320], torch.float32, '688'],
 [4307918848, [320], torch.float32, '686'],
 [4307922944, [320, 320], torch.bfloat16, 'hidden_states5.1_filter_reorderd'],
 [4308127744, [320], torch.bfloat16, 'hidden_states7.1685_bf16'],
 [4308131840, [320], torch.bfloat16, 'hidden_states7.1684_bf16'],
 [4308135936,
  [40, 320],
  torch.bfloat16,
  'input10.1_head_0input10.1_filter_weight_head_0_bf16'],
 [4308164608, [64, 15, 64], torch.bfloat16, 'input10.1_head_0_filter_reorder'],
 [4308287488, [320], torch.bfloat16, 'input10.1_head_0_bias_reorder']]

In [21]:
for i in range(25,35):
    print(i,end=",")
    at = get_info(res[i])

25,tensor(0.0001, dtype=torch.bfloat16) tensor(-0.3398, dtype=torch.bfloat16) tensor(0.2617, dtype=torch.bfloat16)
26,tensor(0.3129) tensor(0.1497) tensor(0.5513)
27,tensor(-0.0150) tensor(-0.3137) tensor(0.3792)
28,tensor(0.0024) tensor(-0.2573) tensor(0.1967)
29,tensor(2.9922e-05, dtype=torch.bfloat16) tensor(-0.2188, dtype=torch.bfloat16) tensor(0.2168, dtype=torch.bfloat16)
30,tensor(0.5977, dtype=torch.bfloat16) tensor(0.2793, dtype=torch.bfloat16) tensor(0.8438, dtype=torch.bfloat16)
31,tensor(0.0002, dtype=torch.bfloat16) tensor(-0.3008, dtype=torch.bfloat16) tensor(0.2812, dtype=torch.bfloat16)
32,tensor(0.0005, dtype=torch.bfloat16) tensor(-0.2012, dtype=torch.bfloat16) tensor(0.2275, dtype=torch.bfloat16)
33,tensor(0.0002, dtype=torch.bfloat16) tensor(-0.3555, dtype=torch.bfloat16) tensor(0.3027, dtype=torch.bfloat16)
34,tensor(0.0014, dtype=torch.bfloat16) tensor(-0.2930, dtype=torch.bfloat16) tensor(0.2275, dtype=torch.bfloat16)


match  

In [ ]:
# shape_map: 
# use bmodel map to match safetensor map 

def search(item):
    # search by shape then get simliary 
    
    pass

In [35]:
# tmp1 = bm.read_tensor_by_address(4294971392, [320, 640], torch.bfloat16, True)
# tmp2 = bm.read_tensor_by_address(4295380992, [320, 640], torch.bfloat16, True)
# tmp = torch.concat([tmp1, tmp2], dim=1)

In [37]:
# torch.allclose(input, other, rtol=1e-05, atol=1e-08, equal_nan=False)
torch.allclose(at.T.float(), data['model.diffusion_model.time_embed.0.weight'].float(), rtol=1e-03, atol=1e-04)
# torch. at.T == data['model.diffusion_model.time_embed.0.weight']

True

In [32]:
len(t)

686

- filter_reorderd
- attention head 

In [56]:
reorder_shape_compute([320, 4, 3, 3])

[1, 320, 1, 288]

9.0

In [ ]:
def filter_reorder(filter_, dtype_type=2):
    shape = filter_.shape
    filter_ = filter_.reshape(-1)
    oc, ic, kh, kw = shape
    IC_PARALLEL = 64 // dtype_type
    kernel_hw = kh * kw
    new_ic = (ic+IC_PARALLEL-1) // IC_PARALLEL
    new_hw = kernel_hw * IC_PARALLEL
    filter_new = np.zeros(oc*new_ic*new_hw, dtype=filter_.dtype)
    for oc_idx in range(oc):
        for ic_idx in range(new_ic):
            for k_idx in range(kernel_hw):
                for inner in range(IC_PARALLEL):
                    if ic_idx * IC_PARALLEL + inner >= ic:
                        break
                    orig_offset = (oc_idx * ic * kh * kw +
                                   (ic_idx * IC_PARALLEL + inner) * kernel_hw + k_idx)
                    trans_offset = (oc_idx * new_ic * new_hw +
                                    ic_idx * new_hw + k_idx * IC_PARALLEL + inner)
                    filter_new[trans_offset] = filter_[orig_offset]
    if new_ic * new_hw > 65535 and len(shape) == 4:
        shape = [1, oc, new_ic, new_hw]
    else:
        shape = [1, oc, 1, new_ic * new_hw]
    filter_new = filter_new.reshape(shape)
    return filter_new